In [ ]:
!pip install autogluon

In [1]:
import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
processed_dataset = pd.read_feather('../data/processed/processed_dataset.feather')
processed_dataset

,age,became_member_on,income,membership_days,gender_F,gender_M,gender_O,age_group,person,offer_id,offer_viewed,offer_completed_after_view,reward,difficulty,duration,offer_type,email,mobile,social,web
0,55.0,2017-07-15,112000.0,376,1,0,0,45-54,0610b486422d4921ae7d2bf64640c50b,9b98b8c7a33c4b65b9aebfe6a799e6d9,False,False,5.0,5.0,7.0,bogo,1.0,1.0,0.0,1.0
1,75.0,2017-05-09,100000.0,443,1,0,0,65+,78afa995795e4d85b5d9ceeca43f5fef,5a8bc65990b245e5a138643cd4eb9837,True,False,0.0,0.0,3.0,informational,1.0,1.0,1.0,0.0
2,75.0,2017-05-09,100000.0,443,1,0,0,65+,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,True,True,5.0,5.0,7.0,bogo,1.0,1.0,0.0,1.0
3,75.0,2017-05-09,100000.0,443,1,0,0,65+,78afa995795e4d85b5d9ceeca43f5fef,ae264e3637204a6fb9bb56bc8210ddfd,True,True,10.0,10.0,7.0,bogo,1.0,1.0,1.0,0.0
4,75.0,2017-05-09,100000.0,443,1,0,0,65+,78afa995795e4d85b5d9ceeca43f5fef,f19421c1d4aa40978ebb69ca19b0e20d,True,False,5.0,5.0,5.0,bogo,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47011,83.0,2016-03-07,50000.0,871,1,0,0,65+,9dc1421481194dcd9400aec7c9ae6366,4d5c57ea9a6940dd891ad53e9dbe8da0,True,True,10.0,10.0,5.0,bogo,1.0,1.0,1.0,1.0
47012,83.0,2016-03-07,50000.0,871,1,0,0,65+,9dc1421481194dcd9400aec7c9ae6366,9b98b8c7a33c4b65b9aebfe6a799e6d9,True,False,5.0,5.0,7.0,bogo,1.0,1.0,0.0,1.0
47013,83.0,2016-03-07,50000.0,871,1,0,0,65+,9dc1421481194dcd9400aec7c9ae6366,ae264e3637204a6fb9bb56bc8210ddfd,True,False,10.0,10.0,7.0,bogo,1.0,1.0,1.0,0.0
47014,62.0,2017-07-22,82000.0,369,1,0,0,55-64,e4052622e5ba45a8b96b59aba68cf068,2298d6c36e964ae4a3e7e9706d1fb8c2,True,True,3.0,7.0,7.0,discount,1.0,1.0,1.0,1.0


In [3]:
target_column = 'offer_completed_after_view'
non_train_features = ['became_member_on', 'age_group', 'person', 'offer_id', 'offer_viewed']

# Remove Nan features from dataset
print(f'Nan features in target: {processed_dataset[target_column].isna().sum()}')
processed_dataset = processed_dataset[processed_dataset[target_column].notna()]

# Remove features not able to use for train
processed_dataset = processed_dataset.drop(columns=non_train_features, axis=1)

# Define the train, validation and test size ratios
train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# Split the dataset into train (70%) and test (30%)
train_dataset, temp_dataset = train_test_split(processed_dataset, test_size=1 - train_ratio, random_state=42)

# Calculate the size ratio of validation and test sets
val_test_ratio = test_ratio / (test_ratio + validation_ratio)

# Split the remaining dataset (X_temp, y_temp) into validation (15%) and test (15%)
val_dataset, test_dataset = train_test_split(temp_dataset, test_size=val_test_ratio, random_state=42)

print('Training set:', train_dataset.shape)
print('Validation set:', val_dataset.shape)
print('Test set:', test_dataset.shape)

Nan features in target: 80
Training set: (32855, 15)
Validation set: (7040, 15)
Test set: (7041, 15)


In [ ]:
# AutoGluon predictor
predictor = TabularPredictor(
    label='offer_completed_after_view',
    problem_type='binary',
    eval_metric='average_precision'  # Use PR_AUC since it is an unbalanced binary classification problem
).fit(
    train_data=train_dataset,
    tuning_data=val_dataset,
    presets='best_quality',
    time_limit=60 * 30, # 30 minutes of time limit
    use_bag_holdout=True,
    verbosity=2,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230519_190014/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "AutogluonModels/ag-20230519_190014/"
AutoGluon Version:  0.7.0
Python Version:     3.8.8
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.4.0: Mon Mar  6 21:00:17 PST 2023; root:xnu-8796.101.5~3/RELEASE_X86_64
Train Data Rows:    32855
Train Data Columns: 14
Tuning Data Rows:    7040
Tuning Data Columns: 14
Label Column: offer_completed_after_view
Preprocessing data ...
Selected class <--> label mapping:  class 1 = True, class 0 = False
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    22238.23 MB
	Train Data (Original)  Memory Usage: 5.87 MB (0.0% of available memory)

In [ ]:
# Note email is dropped because is constant

In [ ]:
# View the summary of the fit
fit_summary = predictor.fit_summary()
display(fit_summary)

In [ ]:
comparison = predictor.leaderboard(val_dataset, silent=True)
comparison